# Generate Website

This notebook generates the website for the dataset.

In [1]:
%reload_ext autoreload
%autoreload 2

## Generate JSON Reports

In [5]:
from depsurf import DepKind, VersionGroup, DepReport, LinuxImage
from typing import Iterator


GROUPS = [VersionGroup.REGULAR, VersionGroup.ARCH, VersionGroup.FLAVOR]

DEPS = [
    DepKind.FUNC("__blk_account_io_done"),
    DepKind.FUNC("__page_cache_release"),
    DepKind.STRUCT("task_struct"),
    DepKind.STRUCT("fs_context"),
    DepKind.STRUCT("inode"),
    # DepKind.FUNC("vfs_rename"),
    # DepKind.FUNC("init_once"),
    # DepKind.FUNC("vfs_read"),
    # DepKind.STRUCT("xfrm_policy_afinfo"),
    # DepKind.STRUCT("xfrm_sec_ctx"),
    DepKind.STRUCT("address_space_operations"),
]

# dep = DEPS[-1]
# DepReport.from_groups(dep, GROUPS).dump_json(dep.report_json_path)
# DepReport.from_dump(dep.report_json_path)

In [25]:
KINDS = [
    # DepKind.FUNC,
    # DepKind.STRUCT,
    # DepKind.LSM,
    DepKind.TRACEPOINT,
]


def get_deps_names(img: LinuxImage, kind: DepKind) -> Iterator[str]:
    if kind == DepKind.FUNC:
        return (func.name for func in img.func_groups.iter_funcs())
    elif kind == DepKind.STRUCT:
        return iter(img.struct_types.data)
    elif kind == DepKind.LSM:
        return iter(img.lsm_hooks)
    elif kind == DepKind.TRACEPOINT:
        return iter(img.tracepoints.data)
    elif kind == DepKind.KFUNC:
        return iter(img.kfuncs)
    else:
        raise ValueError(f"Unknown kind: {kind}")


DEPS = sorted(
    set(
        kind(name)
        for group in GROUPS
        for v in group
        for kind in KINDS
        for name in get_deps_names(v.img, kind)
    )
)

for dep in DEPS:
    report = DepReport.from_groups(dep, GROUPS)
    report.dump_json(dep.report_json_path)

## Generate Markdown Reports

In [20]:
from depsurf import DepReport, DepKind, WEBSITE_PATH

KINDS = [
    # DepKind.FUNC,
    # DepKind.STRUCT,
    # DepKind.LSM,
    # DepKind.TRACEPOINT,
    DepKind.KFUNC,
]


for kind in KINDS:
    paths = (WEBSITE_PATH / kind).rglob("*.json")
    for json_path in paths:
        report = DepReport.from_dump(json_path)
        report.dump_md(report.dep.report_md_path)

## Generate Index Pages

In [24]:
from typing import Iterable
from pathlib import Path

from depsurf import DepKind, VersionGroup, DepReport, WEBSITE_PATH, IssueEnum
from typing import TextIO


KINDS = [
    # DepKind.FUNC,
    # DepKind.STRUCT,
    # DepKind.LSM,
    # DepKind.TRACEPOINT,
    # DepKind.KFUNC,
]

GROUPS = [VersionGroup.REGULAR, VersionGroup.ARCH, VersionGroup.FLAVOR]

ISSUE_SYMBOLS = {
    IssueEnum.ABSENT: "∅",
    IssueEnum.CHANGE: "Δ",
    IssueEnum.SELECTIVE_INLINE: "S",
    IssueEnum.FULL_INLINE: "F",
    IssueEnum.DUPLICATE: "D",
    IssueEnum.COLLISION: "C",
    IssueEnum.TRANSFORMATION: "T",
}

ISSUE_NAMES = {
    IssueEnum.ABSENT: "Absence",
    IssueEnum.CHANGE: "Change",
    IssueEnum.SELECTIVE_INLINE: "Selective Inline",
    IssueEnum.FULL_INLINE: "Full Inline",
    IssueEnum.DUPLICATE: "Duplication",
    IssueEnum.COLLISION: "Collision",
    IssueEnum.TRANSFORMATION: "Transformation",
}


def print_row(report_path: Path, group_only: bool, file: TextIO):
    report = DepReport.from_dump(report_path)
    dep = report.dep
    name = f"[`{dep.name}`]({dep.report_url})"
    if group_only:
        grouped_issues = {group: set() for group in GROUPS}
        for (group, _), issues in report.issues_dict.items():
            grouped_issues[group].update(issues)
        rows = [
            ", ".join([ISSUE_NAMES[e] for e in grouped_issues[group]])
            for group in GROUPS
        ]
    else:
        rows = [
            ",".join([ISSUE_SYMBOLS[e] for e in issues])
            for issues in report.issues_dict.values()
        ]
    print("|".join(["", name, *rows, ""]), file=file)


def print_header(group_only: bool, file: TextIO):
    if group_only:
        col_names = [g for g in GROUPS]
    else:
        col_names = [g.to_str(v) for g in GROUPS for v in g]
    print("| Name | " + " | ".join(col_names) + " |", file=file)
    print("| " + " | ".join(["-"] * (len(col_names) + 1)) + " |", file=file)


def print_index(
    report_paths: Iterable[Path], file_path: Path, group_only: bool = False
):
    with open(file_path, "w") as f:
        kind = file_path.parent.name
        print(f"# {kind}", file=f)
        print_header(group_only, file=f)
        for report_path in sorted(
            report_paths, key=lambda d: d.stem.replace("_", "").lower()
        ):
            print_row(report_path, group_only=group_only, file=f)
    print(f"Saved {file_path}", flush=True)


for kind in KINDS:
    kind_path = WEBSITE_PATH / str(kind)
    subdirs = sorted(d for d in kind_path.iterdir() if d.is_dir())
    if kind in [DepKind.FUNC, DepKind.STRUCT]:
        for subdir in subdirs:
            print_index(
                report_paths=subdir.glob("*.json"),
                file_path=kind_path / f"{subdir.stem}.md",
                group_only=(kind == DepKind.FUNC),
            )
    else:
        print_index(
            report_paths=[p for d in subdirs for p in d.glob("*.json")],
            file_path=kind_path / "index.md",
        )